In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import tqdm as notebook_tqdm
from transformers import AutoModel, AutoTokenizer
from razdel import tokenize
from string import punctuation
import torch
import json

emoj = re.compile('['
                  u'\U0001F600-\U0001F64F'
                  u'\U0001F300-\U0001F5FF'
                  u'\U0001F680-\U0001F6FF'
                  u'\U0001F1E0-\U0001F1FF'
                  u'\U00002500-\U00002BEF'
                  u'\U00002702-\U000027B0'
                  u'\U000024C2-\U0001F251'
                  u'\U0001f926-\U0001f937'
                  u'\U00010000-\U0010ffff'
                  u'\u2640-\u2642'
                  u'\u2600-\u2B55'
                  u'\u200d'
                  u'\u23cf'
                  u'\u23e9'
                  u'\u231a'
                  u'\ufe0f'
                  u'\u3030'
                  ']', re.UNICODE)
punctuation += '—'
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
vectorizer = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")
with open('../DATA/less_semantic_words.txt', 'r', encoding='utf-8') as f:
    stop_words = pd.Series(f.read().split('\n')).unique()

main_href = 'https://vuz.edunetwork.ru/'
usefull_titles = ['Профили', 'Описание', 'Кем работать']

d:\lct-yakut-2023\apps\algorithms\LofDT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def clear_from_stopwords(text, sw):
    text = re.sub(emoj, '', text)
    tokens = list(tokenize(text))
    t = []
    for i in tokens:
        if len(i.text) > 2 and i.text not in sw:
            t.append(i.text)
    return ' '.join(t)

In [4]:
ms = requests.get('https://vuz.edunetwork.ru/specs/').content
soup = BeautifulSoup(ms, 'html.parser')
specials = soup.find_all('li')

In [ ]:
re.sub('\s+', ' ', specials[23].text.replace('\n', ' '))

In [106]:
specials[23].a

<a href="/specs/1">Математика</a>

In [114]:
spec_links = dict()
for i in specials:
    if i: 
        if i.a:
            if 'spec' in i.a['href']:
                spec_links[re.sub('\s+', ' ', i.text.replace('\n', ' '))] = i.a['href']

In [117]:
list(spec_links.keys())[0]

' 01.03.01 — Математика '

In [118]:
soup2 = BeautifulSoup(requests.get(main_href + spec_links[' 01.03.01 — Математика ']).content, 'html.parser')
re.sub(r'\s+', ' ', re.sub(r'\n', '. ', soup2.text))

'. . . . Математика (01.03.01) бакалавриат. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . menu. . . . . . . . . . . . . . Вузы. Колледжи. . . . . . 8 800 302-36-20. . . Подобрать программу. . . . . Войти. Профиль. Поиск. . . . . loop. . . . . . . . . Профиль. Панель вуза. Выйти. . . . . . search. . . brightness_1Вузы. brightness_1Колледжи. . . . . personВойти. personПрофиль. . settingsПанель вуза. . . . exit_to_appВыход. . . . . . . Вузы России Специальности вузов. . . . . . . Математика — бакалавриат (01.03.01) . Укрупненная группа: Математика и механика. . Профили. Алгебра и математическая логикаГеометрия и топологияВычислительная математика и информатикаТеория вероятностей и математическая статистикаМатематическое моделированиеДискретная математикаМатематическая биология и биоинформатикаМатематический анализДифференциальные и интегральные уравненияМатематические методы в экономикеПоказать все. . . Описание. Студенты математического фак

In [135]:
def getData(section):
    d = dict()
    for i in section:
        if i.h2 != None and i.p != None:
            if i.h2.text in usefull_titles:
                d[i.h2.text] = [j.text for j in i.ul.find_all('li')] if i.ul else i.p.text
    return d

In [172]:
soup2.find_all('section')[1]

<section id="profs">
<h2>Профили</h2>
<ul><li>Математическая физика</li><li>Алгебра и математическая логика</li><li>Математический анализ</li><li>Уравнения в частных производных</li><li>Дискретная математика</li><li>Методы вычислений</li><li>Методы оптимизации и оптимального управления</li><li>Теория чисел</li><li>Дифференциальные и интегральные уравнения</li><li>Современные проблемы математики</li><li>Вычислительная математика и информатика</li><li>Математические методы в экономике</li><li>Геометрия и топология</li><li>Математическое моделирование</li><li>Финансовая математика</li></ul><a class="more-link" href="#">Показать все</a>
</section>

In [176]:
spec_desc = dict()
for i in spec_links:
    s = BeautifulSoup(requests.get(main_href + spec_links[i]).content, 'html.parser')
    spec_desc[i] = getData(s.find_all('section'))

In [178]:
with open('../DATA/large_spec_desc.json', 'w', encoding='utf-8') as f:
    json.dump(spec_desc, f)

In [237]:
def vectorization(text, stop_words):
    return vectorizer(
        **tokenizer(
            [
                clear_from_stopwords(text, stop_words)
            ], 
            padding=True, 
            truncation=True, 
            pad_to_multiple_of=65, 
            max_length=65, 
            return_tensors='pt'
        )
    )[0].squeeze(0)


def processingProfiles(profiles):
    d = []
    for i in profiles:
        for j in i.split(' и '):
            d.append(clear_from_stopwords(j, stop_words).lower())
    return d


def processingDescription(desc):
    if type(desc) is list:
        return clear_from_stopwords(' '.join(desc), stop_words).lower()
    return clear_from_stopwords(desc, stop_words).lower()


def processingToBe(desc):
    return clear_from_stopwords(desc, stop_words).lower()


def processingSpec(spec):
    if 'Профили' in spec:
        if len(spec['Профили']) > 0:
            spec['Профили'] = processingProfiles(spec['Профили'])
    if 'Описание' in spec:
        if len(spec['Описание']) > 0:
            spec['Описание'] = processingDescription(spec['Описание'])
    if 'Кем работать' in spec:
        if len(spec['Кем работать']) > 0:
            spec['Кем работать'] = processingToBe(spec['Кем работать'])
    return spec
    

In [238]:
with open('../DATA/large_spec_desc.json', 'r') as f:
    spec_desc2 = json.load(f)
for i in spec_desc2:
    # print(i)
    spec_desc2[i] = processingSpec(spec_desc2[i])

In [235]:
spec_desc2[' 01.03.01 — Математика ']

{'Профили': 'алгебра математическая логика геометрия топология вычислительная математика информатика теория вероятностей математическая статистика математическое моделирование дискретная математика математическая биология биоинформатика математический анализ дифференциальные интегральные уравнения математические методы экономике',
 'Описание': 'студенты математического факультета получат теоретические знания различным математическим предметам алгебра матанализ аналитическая геометрия математическая логика теория вероятностей дискретная математика математическое моделирование дифференциальные уравнения аналитические дисциплины комплексный функциональный анализ входят список теоретического курса однако теорией обучение бакалавров окончится студентов научат полученные знания практике построения алгоритмов разработке компьютерных программ также научатся применять математические методы разных жизненных сферах',
 'Кем работать': 'математика престижной отчасти универсальной специальностью отк

In [234]:
with open('../DATA/large_spec_desc_processed.json', 'w', encoding='utf-8') as f:
    json.dump(spec_desc2, f)